In [1]:
#import future
import rosbag
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

pd.set_option('display.max_columns', 30)

In [2]:
bagPath = r"/home/qmind/Data-2019-10-29/gMitTile_s10_t1.bag"
csvSavePath = bagPath[:-4]+".csv"

In [3]:
# Set the appropriate path to the bag file of interest
"""
# bagPath =r"/home/jack/Desktop/Data-2019-10-29/gMitTile_s05_t1.bag"
bagPath = r"/home/qmind/Data-2019-10-29/gMitTile_s05_t1.bag"
bag = rosbag.Bag(bagPath)
count = 0

imuCount=0
odomCount=0
for topic, msg, t in bag.read_messages():
    #print(t.secs)
    #print(t.nsecs)
    if 'imu' in topic:
        imuCount+=1
        print(msg)
    else:
        odomCount+=1
        #print(topic)
        print(msg)
        #print(msg.header)

    print('\n')
#print('Odom: {}'.format(odomCount))
#print('Imu: {}'.format(imuCount))

bag.close()
"""
pass

In [4]:
# A method to reduce the data to the frequency of the odom topic
# by averages

"""

bag = rosbag.Bag(bagPath)
dfOdom = pd.DataFrame(columns = ['Seq', 'Secs', 'Nanosecs', 'PosX', 'PosY', 'PosZ', 'OrientX', 'OrientY', 'OrientZ', 'OrientW', 'LinX', 'LinY', 'LinZ', 'AngX', 'AngY', 'AngZ', 'AccelAvgX', 'AccelAvgY','AccelAvgZ','AccelDiffX','AccelDiffY','AccelDiffZ'])
count=0
xLA = []
yLA = []
zLA = []
reset = False

for topic, msg, t in bag.read_messages():
#     print("\n\n{}\n".format(topic))
#     print(msg)
    if "imu" in topic:
        if reset:
            reset = False
            xLA = []
            yLA = []
            zLA = []
        xLA.append(msg.linear_acceleration.x)
        yLA.append(msg.linear_acceleration.y)
        zLA.append(msg.linear_acceleration.z)
        
            
        
    if "odom" in topic:
        avgX = np.mean(xLA)
        avgY = np.mean(yLA)
        avgZ = np.mean(zLA)
        if xLA != []:
            diffX = max(xLA) - min(xLA)
            diffY = max(yLA) - min(yLA)
            diffZ = max(zLA) - min(zLA)
        else:
            diffX = None
            diffY = None
            diffZ = None
        
        dfOdom.loc[count] = [msg.header.seq, t.secs, t.nsecs, msg.pose.pose.position.x, msg.pose.pose.position.y, msg.pose.pose.position.z, msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, msg.pose.pose.orientation.z, msg.pose.pose.orientation.w, msg.twist.twist.linear.x, msg.twist.twist.linear.y, msg.twist.twist.linear.z, msg.twist.twist.angular.x, msg.twist.twist.angular.y, msg.twist.twist.angular.z, avgX, avgY, avgZ, diffX, diffY, diffZ]
        reset = True
            
        
    count+=1
    
bag.close()

"""
pass

In [5]:
# create two dataframes, one for IMU data and one for Odometry data


bag = rosbag.Bag(bagPath)
odomDf = pd.DataFrame(columns = ['Seq', 'Secs', 'Nanosecs', 
                                 'OdomPosX', 'OdomPosY', 'OdomPosZ', 
                                 'OdomOrientX', 'OdomOrientY', 'OdomOrientZ', 'OdomOrientW', 
                                 'OdomLinX', 'OdomLinY', 'OdomLinZ', 
                                 'OdomAngX', 'OdomAngY', 'OdomAngZ'])

imuDf = pd.DataFrame(columns = ['Seq', 'Secs', 'Nanosecs', 
                                'ImuOrientX', 'ImuOrientY', 'ImuOrientZ', 'ImuOrientW', 
                                'ImuAngVelX', 'ImuAngVelY', 'ImuAngVelZ', 
                                'ImuAccelX', 'ImuAccelY', 'ImuAccelZ'] )
count = 0
imuCount = 0
odomCount = 0
for topic, msg, t in bag.read_messages():

    if topic == "/imu":
        imuDf.loc[imuCount] = [msg.header.seq, t.secs, t.nsecs, 
                               msg.orientation.x, msg.orientation.y, msg.orientation.z, msg.orientation.w, 
                               msg.angular_velocity.x, msg.angular_velocity.y, msg.angular_velocity.z, 
                               msg.linear_acceleration.x, msg.linear_acceleration.y, msg.linear_acceleration.z]
        imuCount+=1
    
        
    if topic == '/odom':
        odomDf.loc[odomCount] = [msg.header.seq, t.secs, t.nsecs, msg.pose.pose.position.x, msg.pose.pose.position.y, 
                             msg.pose.pose.position.z, msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, 
                             msg.pose.pose.orientation.z, msg.pose.pose.orientation.w, msg.twist.twist.linear.x, 
                             msg.twist.twist.linear.y, msg.twist.twist.linear.z, msg.twist.twist.angular.x, 
                             msg.twist.twist.angular.y, msg.twist.twist.angular.z]
        odomCount+=1
    count+=1
            
print("Total data points: {}".format(count))
print("IMU data points: {}".format(imuCount))
print("Odometry data points: {}".format(odomCount))
bag.close()

Total data points: 7877
IMU data points: 6515
Odometry data points: 1362


In [6]:
startTime = min(imuDf.Secs[0], odomDf.Secs[0])
startTime

1572371662.0

In [7]:
# start at 0 seconds
odomDf.Secs = odomDf.Secs - startTime
imuDf.Secs = imuDf.Secs - startTime

In [8]:
#make a time column from secs and nanosecs
odomDf['Time'] = odomDf.Secs + odomDf.Nanosecs/10**(9)
imuDf['Time'] = imuDf.Secs + imuDf.Nanosecs/10**(9)

In [9]:
# move time column to front
odomDf = odomDf[[odomDf.columns.tolist()[-1]] + odomDf.columns.tolist()[:-1]]
imuDf = imuDf[[imuDf.columns.tolist()[-1]] + imuDf.columns.tolist()[:-1]]

In [10]:
# sanity check for errors
print(type(odomDf.Nanosecs[0]))
print(odomDf.Time[1] - odomDf.Time[0])
print((odomDf.Nanosecs[1]-odomDf.Nanosecs[0])/10**9)

<type 'numpy.float64'>
0.029181991000000004
0.029181991


In [11]:
# remove un-needed cols
odomDf = odomDf.drop(columns = ['Secs', 'Nanosecs', 'Seq'])
imuDf = imuDf.drop(columns = ['Secs', 'Nanosecs', 'Seq'])

In [12]:
# zero start time again because nanoseconds werent zeroed the first time
startTime2 = min(imuDf.Time[0], odomDf.Time[0])
odomDf.Time = odomDf.Time - startTime2
imuDf.Time = imuDf.Time - startTime2

In [13]:
odomDf['Sensor'] = 'Odom'
imuDf['Sensor'] = 'Imu'

In [14]:
imuDf.head(10)

,Time,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,Sensor
0,0.000778,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992,Imu
1,0.005144,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992,Imu
2,0.009503,-0.000573,-0.017395,-0.817924,0.575055,0.035119,0.007449,0.009578,0.556652,0.246603,10.154408,Imu
3,0.014124,-0.000178,-0.016802,-0.817935,0.575058,0.025541,0.011706,0.000000,0.036512,0.174178,9.561244,Imu
4,0.021683,0.000473,-0.016277,-0.817967,0.575028,0.026605,0.038311,-0.010642,0.079607,0.246004,9.926360,Imu
5,0.035865,0.001404,-0.016318,-0.817998,0.574980,0.026605,0.056403,-0.013835,0.134674,0.357335,11.064204,Imu
6,0.040598,0.001506,-0.015737,-0.818019,0.574966,0.012770,0.053210,-0.009578,0.164003,0.284311,11.574169,Imu
7,0.059297,0.000648,-0.018033,-0.818087,0.574804,-0.013835,0.020220,-0.013835,0.307056,0.401627,11.410167,Imu
8,0.063450,0.000334,-0.018426,-0.818114,0.574753,-0.026605,0.000000,-0.023412,0.536900,0.457891,10.324995,Imu
9,0.067656,0.000334,-0.018426,-0.818114,0.574753,-0.026605,0.000000,-0.023412,0.536900,0.457891,10.324995,Imu


In [15]:
odomDf.head(10)

,Time,OdomPosX,OdomPosY,OdomPosZ,OdomOrientX,OdomOrientY,OdomOrientZ,OdomOrientW,OdomLinX,OdomLinY,OdomLinZ,OdomAngX,OdomAngY,OdomAngZ,Sensor
0,0.000000,10.216013,-2.734961,0.0,0.0,0.0,-0.105752,0.994393,0.099009,0.0,0.0,0.0,0.0,0.012734,Odom
1,0.029182,10.219403,-2.735690,0.0,0.0,0.0,-0.105754,0.994392,0.101987,0.0,0.0,0.0,0.0,-0.000144,Odom
2,0.050881,10.222570,-2.736372,0.0,0.0,0.0,-0.105980,0.994368,0.098175,0.0,0.0,0.0,0.0,-0.013781,Odom
3,0.094341,10.225861,-2.737082,0.0,0.0,0.0,-0.106235,0.994341,0.099009,0.0,0.0,0.0,0.0,-0.015027,Odom
4,0.128224,10.229076,-2.737777,0.0,0.0,0.0,-0.106192,0.994346,0.099709,0.0,0.0,0.0,0.0,0.002601,Odom
5,0.154153,10.232367,-2.738488,0.0,0.0,0.0,-0.106132,0.994352,0.099009,0.0,0.0,0.0,0.0,0.003541,Odom
6,0.188922,10.235706,-2.739209,0.0,0.0,0.0,-0.106111,0.994354,0.100498,0.0,0.0,0.0,0.0,0.001248,Odom
7,0.223885,10.238972,-2.739914,0.0,0.0,0.0,-0.106090,0.994357,0.098265,0.0,0.0,0.0,0.0,0.001213,Odom
8,0.263538,10.242213,-2.740613,0.0,0.0,0.0,-0.105970,0.994369,0.097521,0.0,0.0,0.0,0.0,0.007135,Odom
9,0.303034,10.245578,-2.741339,0.0,0.0,0.0,-0.106011,0.994365,0.101243,0.0,0.0,0.0,0.0,-0.002444,Odom


In [16]:
allDataDf = pd.concat([odomDf, imuDf], axis=0, sort=False)
allDataDf = allDataDf.sort_values('Time').reset_index()
allDataDf = allDataDf.drop(columns='index')


In [17]:
allDataDf.Time[1]

0.0007779480000000005

In [18]:
allDataDf.head(10)

,Time,OdomPosX,OdomPosY,OdomPosZ,OdomOrientX,OdomOrientY,OdomOrientZ,OdomOrientW,OdomLinX,OdomLinY,OdomLinZ,OdomAngX,OdomAngY,OdomAngZ,Sensor,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ
0,0.000000,10.216013,-2.734961,0.0,0.0,0.0,-0.105752,0.994393,0.099009,0.0,0.0,0.0,0.0,0.012734,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992
2,0.005144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992
3,0.009503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000573,-0.017395,-0.817924,0.575055,0.035119,0.007449,0.009578,0.556652,0.246603,10.154408
4,0.014124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000178,-0.016802,-0.817935,0.575058,0.025541,0.011706,0.000000,0.036512,0.174178,9.561244
5,0.021683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.000473,-0.016277,-0.817967,0.575028,0.026605,0.038311,-0.010642,0.079607,0.246004,9.926360
6,0.029182,10.219403,-2.735690,0.0,0.0,0.0,-0.105754,0.994392,0.101987,0.0,0.0,0.0,0.0,-0.000144,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.035865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.001404,-0.016318,-0.817998,0.574980,0.026605,0.056403,-0.013835,0.134674,0.357335,11.064204
8,0.040598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.001506,-0.015737,-0.818019,0.574966,0.012770,0.053210,-0.009578,0.164003,0.284311,11.574169
9,0.050881,10.222570,-2.736372,0.0,0.0,0.0,-0.105980,0.994368,0.098175,0.0,0.0,0.0,0.0,-0.013781,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
allDataDf.to_csv(csvSavePath)

In [20]:
checkDf = pd.read_csv(csvSavePath)

In [21]:
checkDf.head(10)

,Unnamed: 0,Time,OdomPosX,OdomPosY,OdomPosZ,OdomOrientX,OdomOrientY,OdomOrientZ,OdomOrientW,OdomLinX,OdomLinY,OdomLinZ,OdomAngX,OdomAngY,OdomAngZ,Sensor,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ
0,0,0.000000,10.216013,-2.734961,0.0,0.0,0.0,-0.105752,0.994393,0.099009,0.0,0.0,0.0,0.0,0.012734,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.000778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992
2,2,0.005144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000041,-0.016951,-0.817958,0.575021,0.031926,0.000000,0.015963,0.428562,0.454898,10.986992
3,3,0.009503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000573,-0.017395,-0.817924,0.575055,0.035119,0.007449,0.009578,0.556652,0.246603,10.154408
4,4,0.014124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,-0.000178,-0.016802,-0.817935,0.575058,0.025541,0.011706,0.000000,0.036512,0.174178,9.561244
5,5,0.021683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.000473,-0.016277,-0.817967,0.575028,0.026605,0.038311,-0.010642,0.079607,0.246004,9.926360
6,6,0.029182,10.219403,-2.735690,0.0,0.0,0.0,-0.105754,0.994392,0.101987,0.0,0.0,0.0,0.0,-0.000144,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,0.035865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.001404,-0.016318,-0.817998,0.574980,0.026605,0.056403,-0.013835,0.134674,0.357335,11.064204
8,8,0.040598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Imu,0.001506,-0.015737,-0.818019,0.574966,0.012770,0.053210,-0.009578,0.164003,0.284311,11.574169
9,9,0.050881,10.222570,-2.736372,0.0,0.0,0.0,-0.105980,0.994368,0.098175,0.0,0.0,0.0,0.0,-0.013781,Odom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
checkDf.shape

(7877, 26)